In [26]:
import pandas as pd
import sqlite3
import openpyxl
import requests
from io import BytesIO

In [27]:
# Verbindung zur SQLite-Datenbank herstellen
conn = sqlite3.connect("letsmeet.db")
cursor = conn.cursor()

In [28]:
# Tabelle erstellen (falls nicht vorhanden)
cursor.execute("""
CREATE TABLE IF NOT EXISTS users_from_excel (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    age INTEGER,
    city TEXT
);
""")

In [32]:
# Raw-URL der Excel-Datei
excel_url = "https://raw.githubusercontent.com/Indorhc3/LetsMeet/main/Lets%20Meet%20DB%20Dump.xlsx"

# Excel-Datei aus der URL herunterladen und einlesen
response = requests.get(excel_url)
response.raise_for_status()  # Sicherstellen, dass der Download erfolgreich war
df = pd.read_excel(BytesIO(response.content))

In [33]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS users_from_excel (
    name TEXT,
    address TEXT,
    phone TEXT,
    hobbies TEXT,
    email TEXT,
    gender TEXT,
    interests TEXT,
    birthdate TEXT
);
""")
conn.commit()

In [35]:
cursor.execute("ALTER TABLE users_from_excel ADD COLUMN address TEXT;")
cursor.execute("ALTER TABLE users_from_excel ADD COLUMN phone TEXT;")
cursor.execute("ALTER TABLE users_from_excel ADD COLUMN hobbies TEXT;")
cursor.execute("ALTER TABLE users_from_excel ADD COLUMN email TEXT;")
cursor.execute("ALTER TABLE users_from_excel ADD COLUMN gender TEXT;")
cursor.execute("ALTER TABLE users_from_excel ADD COLUMN interests TEXT;")
cursor.execute("ALTER TABLE users_from_excel ADD COLUMN birthdate TEXT;")
conn.commit()

In [36]:
for _, row in df.iterrows():
    cursor.execute("""
    INSERT INTO users_from_excel (
        name, 
        address, 
        phone, 
        hobbies, 
        email, 
        gender, 
        interests, 
        birthdate
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        row['Nachname, Vorname'], 
        row['Straße Nr, PLZ Ort'], 
        row['Telefon'], 
        row['Hobby1 %Prio1%; Hobby2 %Prio2%; Hobby3 %Prio3%; Hobby4 %Prio4%; Hobby5 %Prio5%;'], 
        row['E-Mail'], 
        row['Geschlecht (m/w/nonbinary)'], 
        row['Interessiert an'], 
        row['Geburtsdatum']
    ))
conn.commit()

In [38]:
cursor.execute("SELECT * FROM users_from_excel;")
rows = cursor.fetchall()
for row in rows:
    print(row)


(1, 'Forster, Martin', None, None, 'Minslebener Str. 0, 46286, Dorsten', '02372 8020', 'Fremdsprachenkenntnisse erweitern %78%; Im Wasser waten %80%; Schwierige Probleme klären %61%; Morgens Früh aufstehen %17%; ', 'martin.forster@web.ork', 'm', 'w', '07.03.1959')
(2, 'Elina, Tsanaklidou', None, None, 'Gartenweg 13, 69126, Heidelberg', '06221 / 98689', 'Jemanden massieren %57%; Mir die Probleme von anderen anhören %21%; Abends seinem Partner Ereignisse des Tages erzählen %53%; Für einen guten Zweck spenden %25%; ', 'tsanaklidou.elina@1mal1.te', 'w', 'm', '28.02.1958')
(3, 'Vildan, şahin', None, None, 'Heinrich-Heine-Straße 99c, 17489, Greifswald', '03834 / 22951', 'In der Stadt herumbummeln %94%; Mit Freunden zusammen sein %20%; Tagebuch schreiben %12%; Ausschlafen %72%; ', 'şahin.vildan@gmaiil.ork', 'w', 'm', '27.07.1996')
(4, 'Bäumker, Ellen', None, None, 'Weidenring 39, 81539, München', '0162 / 249788', 'An einer Gruppenreise teilnehmen %97%; Seine finanziellen Angelegenheiten regel